In [1]:
from utility_bfvariater import *
import utility_common as common

# creat a BF variater
er = BFVariater()

# 1. Central Value

In [2]:
def saveCentralValues(er):
    print( "save conteral values")

    contralValue = []
    for i in range(4):

        ndata = er.ndata[i]
        nmcbg = er.nmcbg[i].dot(np.ones(3))
        nfake = er.nfake[i]
        a = np.sum(er.a[i] * er.ca, axis=1)

        slv0 = BFSolver3D(a)
        br0 = slv0.solveQuadEqn(slv0.setMeasuredX(nData=ndata, nMcbg=nmcbg+nfake))
        contralValue.append(br0)
    contralValue = np.array(contralValue).reshape(12)

    baseDir = common.getBaseDirectory() 

    print(contralValue)

    np.save(baseDir+'/data/combine/contralValue',contralValue)
saveCentralValues(er)

save conteral values
[0.11000775 0.10867409 0.10036348 0.11241179 0.11032106 0.10078176
 0.10537486 0.10834266 0.10156052 0.10917631 0.10996087 0.10180731]


# 2. uncertainty

### 2.1 Statistical Uncertainty

In [3]:
def saveStatError(er):
    print( "relative statistical uncertainty [%]")

    stat_sigma, stat_covar = [],[]

    for source in ["data","mcbg","mcsg"]:
        sigma, variance_raw = er.errStat(source,returnCovar=True)
        # reshape 4x3x3 to 12x12
        variance = np.zeros([12,12])
        for c in range(4):
            for i in range(3):
                for j in range(3):
                    variance[c*3+i,c*3+j] = variance_raw[c,i,j]
                    
        stat_sigma.append(sigma)
        stat_covar.append(variance)

    stat_sigma = np.array(stat_sigma)  
    stat_covar = np.array(stat_covar) 

    er.io_printErrorForExcelFormat(stat_sigma.transpose(1,0,2))

    s = stat_sigma.shape
    stat_sigma = stat_sigma.reshape(s[0],s[1]*s[2])

    baseDir = common.getBaseDirectory() 
    np.save(baseDir+'/data/combine/stat_sigma',stat_sigma)
    np.save(baseDir+'/data/combine/stat_covar',stat_covar)

saveStatError(er)

relative statistical uncertainty [%]
0.554,0.517,1.223, 0.983,0.870,2.030, 0.711,0.552,1.454, 1.235,0.969,2.466
0.136,0.547,0.607, 0.142,0.517,0.647, 0.639,0.180,0.710, 0.595,0.179,0.732
0.266,0.227,0.572, 0.464,0.394,0.929, 0.323,0.265,0.695, 0.571,0.452,1.128


### 2.2 Systematical Uncertainty

In [4]:
def saveSystError(er):
    print( "relative systematics uncertainty [%]")
    syst_sigma, syst_covar = [],[]

    for source in ["BtmUp","BteUp"]:
        sigma = er.errConstent(source)
        variance = np.outer(sigma.reshape(1,12),sigma.reshape(1,12))
        syst_sigma.append(sigma)
        syst_covar.append(variance)
        
    for source in ["lumin","mctt","mctw","mcw",'mcz','mcvv',
                   "fakee","fakemu","fakeetau","fakemutau"]:
        sigma = er.errSystem_crossSection(source)
        variance = np.outer(sigma.reshape(1,12),sigma.reshape(1,12))
        syst_sigma.append(sigma)
        syst_covar.append(variance)
        
        
    for source in ["TopPtReweight","ERecoEff","EIDEff","MuRecoEff","MuIDEff","TauIDEff","JetToTauIDEff",
                   "EPt","MuPt","Tau0Pt","Tau1Pt","Tau10Pt",
                  ]:
        sigma = er.errSystem_downVariation(source)
        variance = np.outer(sigma.reshape(1,12),sigma.reshape(1,12))
        syst_sigma.append(sigma)
        syst_covar.append(variance)
        
    for source in ["TauReweight1000","TauReweight11000","TauReweight21000",
                   "TauReweight3000","TauReweight13000"
                  ]:
        sigma = er.errSystem_nominalDownVariation(source)
        variance = np.outer(sigma.reshape(1,12),sigma.reshape(1,12))
        syst_sigma.append(sigma)
        syst_covar.append(variance)
        
        
    for source in ["Pileup","JES","JER","BTag","Mistag"]:
        sigma = er.errSystem_upDownVariation(source)
        variance = np.outer(sigma,sigma)
        syst_sigma.append(sigma)
        syst_covar.append(variance)        
    
    syst_sigma = np.array(syst_sigma) 
    syst_covar = np.array(syst_covar) 

    er.io_printErrorForExcelFormat(syst_sigma.transpose(1,0,2))

    s = syst_sigma.shape
    syst_sigma = syst_sigma.reshape(s[0],s[1]*s[2])

    baseDir = common.getBaseDirectory() 
    np.save(baseDir+'/data/combine/syst_sigma',syst_sigma)
    np.save(baseDir+'/data/combine/syst_covar',syst_covar)

saveSystError(er)

relative systematics uncertainty [%]
0.005,0.050,0.075, 0.005,0.048,0.074, 0.007,0.048,0.074, 0.007,0.048,0.074
0.028,0.010,0.059, 0.028,0.010,0.058, 0.024,0.007,0.058, 0.024,0.008,0.059
0.150,0.267,0.033, 0.048,0.072,0.007, 0.136,0.140,0.073, 0.007,0.048,0.009
0.010,0.002,0.105, 0.002,0.011,0.052, 0.007,0.006,0.119, 0.001,0.000,0.007
0.011,0.002,0.110, 0.002,0.012,0.054, 0.007,0.007,0.124, 0.001,0.000,0.007
0.156,0.156,0.147, 0.052,0.051,0.040, 0.149,0.153,0.165, 0.048,0.048,0.074
0.290,1.361,0.152, 0.088,0.383,0.047, 0.828,0.255,0.028, 0.118,0.094,0.110
0.001,0.020,0.011, 0.001,0.008,0.004, 0.017,0.001,0.008, 0.006,0.001,0.001
0.000,0.000,0.000, 0.000,0.000,0.000, 0.336,0.336,0.729, 0.177,0.173,0.322
0.617,0.590,1.285, 0.401,0.381,0.700, 0.000,0.000,0.000, 0.000,0.000,0.000
0.000,0.000,0.000, 0.000,0.000,0.000, 0.025,0.034,0.184, 0.013,0.018,0.098
0.028,0.038,0.195, 0.016,0.021,0.107, 0.000,0.000,0.000, 0.000,0.000,0.000
0.006,0.009,0.035, 0.006,0.010,0.017, 0.006,0.007,0.040, 0.006,

# 3. output Latex Table

In [4]:
%run -i '/Users/zihengchen/Documents/Analysis/workplace/scripts/note_uncertainty.py'

In [ ]:
#     for source in ["e","mu","tauID",'jetMisTauID']:
#         sigma = er.errSystem_objectEff(source)
#         variance = np.outer(sigma.reshape(1,12),sigma.reshape(1,12))
#         syst_sigma.append(sigma)
#         syst_covar.append(variance)